In [ ]:
# Trying Optical Character Recognition (OCR)
# reading the text from scanned pdf
import pytesseract # OCR tool for python
from pdf2image import convert_from_path
import xlsxwriter # excel-related module
from tqdm import tqdm # arabic word meaning 'progress'

images=convert_from_path('D:\\on campus job\\test.pdf')
excel=xlsxwriter.Workbook('D:\\on campus job\\Book1.xlsx')
sheet=excel.add_worksheet()

for i in tqdm(range(10)):
    image=images[i]
    text=pytesseract.image_to_string(image,lang='jpn')
    column=0
    for item in text.splitlines():
        sheet.write(i,column, item)
        column+=1

excel.close() #11mins, low efficient

In [ ]:
# separating sheets into sole files from excel files
# each excel file has many sheets.
import pandas as pd
import os #operating system 
from glob import glob # pathname pattern expansion
from tqdm.auto import tqdm # arabic word meaning 'progress'

path='dir'+'Otsuchi Bay CTD_Hideki Fukuda (2011-2023)\\Otsuchi Bay CTD_Hideki Fukuda (2011-2023)\\'
folders=sorted(glob(path+'2*'))
for k in tqdm(range(len(folders))):
    files=os.listdir(folders[k])
    for j in range(len(files)):
        excel=pd.ExcelFile(folders[k]+'\\'+files[j])
        i=0 # in some excel files, first sheet is the location info
        if excel.sheet_names[i]!='Location':
            df=pd.read_excel(folders[k]+'\\'+files[j],sheet_name=i)
            name=files[j][:-5]+excel.sheet_names[i]+'.csv'
            new=name.replace(" ","") #string method
            new2=new.replace("'","p") #string method
            df.to_csv(path+'Separated files\\'+new2)
        for i in range(1,len(excel.sheet_names)):
            df=pd.read_excel(folders[k]+'\\'+files[j],sheet_name=i)
            name=files[j][:-5]+excel.sheet_names[i]+'.csv'
            new=name.replace(" ","")
            new2=new.replace("'","p")
            df.to_csv(path+'Separated files\\'+new2)


In [ ]:
# extracting the station name, latitude, longitude, and writing them into excel

import pandas as pd
import os
import xlsxwriter
path='dir'+'Otsuchi Bay CTD_Hideki Fukuda (2011-2023)\\Otsuchi Bay CTD_Hideki Fukuda (2011-2023)\\'
files=os.listdir(path+"Separated files")
df=pd.read_excel(path+'Summary of station name and location in Otsuchi Bay.xlsx')
df.iloc[5,0]='1p';df.iloc[7,0]='2p'
df=df.astype({'Unnamed: 0':'string'}) # convert string-int mix into string

book=xlsxwriter.Workbook(path+'x.xlsx')
sh=book.add_worksheet()

for i in range(len(files)):
    try:
        if files[i].find('Ex.1')!=-1:
            SN='Ex.1'
        elif files[i].find('Ex1')!=-1:
            SN='Ex1'
        elif files[i].find('Sta.St2')!=-1:
            SN='2'
        else:
            SN=files[i].split('.')[-2] #station name
            if SN.find('(')!=-1: # some stations have multiple samplings
                SN=SN[:-3]
            if len(SN)>4: # some long string
                SN=SN[-2:]
            if SN.find('(採泥)')!=-1:# kanji undetectable
                SN=SN[:-4]
            if SN.find('_採泥')!=-1:
                SN=SN[:-3]
            
        Df=df.loc[df.iloc[:,0]==SN] # match the station
        lon=df.iloc[1,2]+str(Df.iloc[0,2])
        lat=df.iloc[1,1]+str(Df.iloc[0,1]);lat=lat.replace(" ","")
        Data=pd.read_csv(path+"Separated files\\"+files[i])# open the STD file
        date=Data.iloc[11,1][-10:]
        time=Data.iloc[12,1][-8:]
        if date.find('/')==-1: # -1 means the character not existed; data in different format
            date=Data.iloc[13,1][-19:-9]
            time=Data.iloc[13,1][-8:]
        
        sh.write(i,0,date);sh.write(i,1,time);sh.write(i,2,lat);sh.write(i,3,lon)
    except:sh.write(i,0,files[i]);continue
book.close() # it is necessary
